In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

/Users/fytang/workSpace/miniconda3/envs/tf114/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/fytang/workSpace/miniconda3/envs/tf114/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/fytang/workSpace/miniconda3/envs/tf114/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/fytang

In [2]:
tf.logging.set_verbosity(tf.logging.INFO)
mnist = input_data.read_data_sets("./MNIST_data",one_hot=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


使用estimator自定义lenet模型

In [3]:
def lenet(x,is_training):
    x = tf.reshape(x,shape=[-1,28,28,1])
    net =tf.layers.conv2d(x,32,5,activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(net,2,2)
    net = tf.layers.conv2d(net,64,3,activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(net,2,2)
    net = tf.contrib.layers.flatten(net)
    net = tf.layers.dense(net,1024)
    net = tf.layers.dropout(net,rate=0.4,training=is_training)
    return tf.layers.dense(net,10)

#### 自定义estimator中使用的模型，定义的函数有4个输入：
features给出了在输入函数中会提供的输入层张量，注意这是一个字典，
字典里的内容是通过tf.estimator.inputs.numpy_input_fn中x参数指定，
labels是正确答案，这个字段内容通过numpy_input_fn中的y参数给出。
mode的去只有三种可能，分别对应train，eval，predict。
params是一个字典，存放模型相关的任意超参数，eg：lr


In [6]:
def model_fn(features,labels,mode,params):
    predict = lenet(
        features["image"],mode == tf.estimator.ModeKeys.TRAIN
    )
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions={"result":tf.argmax(predict,1)}
        )
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=predict,labels=labels))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=params["learning_rate"])
    train_op = optimizer.minimize(
        loss = loss,global_step=tf.train.get_global_step()
    )
    eval_metric_ops = {
        "my_metric":tf.metrics.accuracy(
            tf.argmax(predict,1),labels
        )
    }
    
    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss = loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops
    )

In [9]:
model_params = {"learning_rate":0.01}

In [13]:
estimator = tf.estimator.Estimator(model_fn=model_fn,params=model_params,model_dir = "./udf_log")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './udf_log', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe78c61af60>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [14]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"image":mnist.train.images},
    y=mnist.train.labels.astype(np.int32),
    num_epochs=None,
    batch_size=128,
    shuffle=True
)

In [15]:
estimator.train(input_fn=train_input_fn,steps=30000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


Instructions for updating:
Use keras.layers.dropout instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into ./udf_log/model.ckpt.
INFO:tensorflow:loss = 2.3241022, step = 1
INFO:tensorflow:global_step/sec: 18.9932
INFO:tensorflow:loss = 1.7296016, step = 101 (5.266 sec)
INFO:tensorflow:global_step/sec: 20.3703
INFO:tensorflow:loss = 0.676988, step = 201 (4.909 sec)
INFO:tensorflow:global_step/sec: 20.289
INFO:tensorflow:loss = 0.49471512, step = 301 (4.929 sec)
INFO:tensorflow:global_step/sec: 20.0317
INFO:tensorflow:loss = 0.40507102, step = 401 (4.992 sec)
INFO:tensorflow:global_step/sec: 20.1733
INFO:tensorflow

INFO:tensorflow:global_step/sec: 20.2497
INFO:tensorflow:loss = 0.14714053, step = 4601 (4.938 sec)
INFO:tensorflow:global_step/sec: 20.1729
INFO:tensorflow:loss = 0.069345266, step = 4701 (4.957 sec)
INFO:tensorflow:global_step/sec: 19.7504
INFO:tensorflow:loss = 0.09245147, step = 4801 (5.063 sec)
INFO:tensorflow:global_step/sec: 20.2458
INFO:tensorflow:loss = 0.14516214, step = 4901 (4.939 sec)
INFO:tensorflow:global_step/sec: 19.9145
INFO:tensorflow:loss = 0.03937383, step = 5001 (5.021 sec)
INFO:tensorflow:global_step/sec: 20.2138
INFO:tensorflow:loss = 0.1473541, step = 5101 (4.947 sec)
INFO:tensorflow:global_step/sec: 19.9801
INFO:tensorflow:loss = 0.077535704, step = 5201 (5.005 sec)
INFO:tensorflow:global_step/sec: 20.2362
INFO:tensorflow:loss = 0.038369354, step = 5301 (4.942 sec)
INFO:tensorflow:global_step/sec: 19.8051
INFO:tensorflow:loss = 0.021827865, step = 5401 (5.049 sec)
INFO:tensorflow:global_step/sec: 20.1445
INFO:tensorflow:loss = 0.098043755, step = 5501 (4.964 s

INFO:tensorflow:global_step/sec: 19.6764
INFO:tensorflow:loss = 0.13373329, step = 12701 (5.082 sec)
INFO:tensorflow:global_step/sec: 19.3018
INFO:tensorflow:loss = 0.022661239, step = 12801 (5.181 sec)
INFO:tensorflow:global_step/sec: 19.2993
INFO:tensorflow:loss = 0.03457316, step = 12901 (5.182 sec)
INFO:tensorflow:global_step/sec: 19.3346
INFO:tensorflow:loss = 0.052166916, step = 13001 (5.172 sec)
INFO:tensorflow:global_step/sec: 19.4369
INFO:tensorflow:loss = 0.022655964, step = 13101 (5.145 sec)
INFO:tensorflow:global_step/sec: 19.8365
INFO:tensorflow:loss = 0.17919725, step = 13201 (5.041 sec)
INFO:tensorflow:global_step/sec: 19.7777
INFO:tensorflow:loss = 0.03837707, step = 13301 (5.056 sec)
INFO:tensorflow:global_step/sec: 19.3575
INFO:tensorflow:loss = 0.05598544, step = 13401 (5.166 sec)
INFO:tensorflow:global_step/sec: 19.1845
INFO:tensorflow:loss = 0.045968078, step = 13501 (5.213 sec)
INFO:tensorflow:global_step/sec: 19.3103
INFO:tensorflow:loss = 0.027033634, step = 136

INFO:tensorflow:global_step/sec: 18.8307
INFO:tensorflow:loss = 0.051864725, step = 20801 (5.310 sec)
INFO:tensorflow:global_step/sec: 19.2815
INFO:tensorflow:loss = 0.019904729, step = 20901 (5.186 sec)
INFO:tensorflow:global_step/sec: 19.3111
INFO:tensorflow:loss = 0.02684717, step = 21001 (5.178 sec)
INFO:tensorflow:global_step/sec: 19.3097
INFO:tensorflow:loss = 0.031701762, step = 21101 (5.179 sec)
INFO:tensorflow:global_step/sec: 19.3428
INFO:tensorflow:loss = 0.023589652, step = 21201 (5.170 sec)
INFO:tensorflow:global_step/sec: 19.3191
INFO:tensorflow:loss = 0.014757856, step = 21301 (5.176 sec)
INFO:tensorflow:global_step/sec: 18.866
INFO:tensorflow:loss = 0.01553615, step = 21401 (5.301 sec)
INFO:tensorflow:global_step/sec: 19.1707
INFO:tensorflow:loss = 0.07221767, step = 21501 (5.216 sec)
INFO:tensorflow:global_step/sec: 19.3123
INFO:tensorflow:loss = 0.027856361, step = 21601 (5.178 sec)
INFO:tensorflow:global_step/sec: 19.2918
INFO:tensorflow:loss = 0.023570567, step = 21

INFO:tensorflow:global_step/sec: 19.3045
INFO:tensorflow:loss = 0.011690335, step = 28801 (5.180 sec)
INFO:tensorflow:global_step/sec: 19.2274
INFO:tensorflow:loss = 0.043803737, step = 28901 (5.201 sec)
INFO:tensorflow:global_step/sec: 18.4905
INFO:tensorflow:loss = 0.01736525, step = 29001 (5.408 sec)
INFO:tensorflow:global_step/sec: 18.8974
INFO:tensorflow:loss = 0.038974263, step = 29101 (5.292 sec)
INFO:tensorflow:global_step/sec: 19.1782
INFO:tensorflow:loss = 0.04259716, step = 29201 (5.214 sec)
INFO:tensorflow:global_step/sec: 19.312
INFO:tensorflow:loss = 0.03443233, step = 29301 (5.178 sec)
INFO:tensorflow:global_step/sec: 19.4032
INFO:tensorflow:loss = 0.04709986, step = 29401 (5.154 sec)
INFO:tensorflow:global_step/sec: 19.3495
INFO:tensorflow:loss = 0.0036865347, step = 29501 (5.168 sec)
INFO:tensorflow:global_step/sec: 18.9277
INFO:tensorflow:loss = 0.049088333, step = 29601 (5.283 sec)
INFO:tensorflow:global_step/sec: 18.8688
INFO:tensorflow:loss = 0.01912629, step = 297

In [16]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"image":mnist.test.images},
    y=mnist.test.labels.astype(np.int32),
    num_epochs=1,
    batch_size=128,
    shuffle=False
)

In [17]:
test_result = estimator.evaluate(input_fn=test_input_fn)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Starting evaluation at 2022-06-17T01:58:06Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./udf_log/model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2022-06-17-01:58:07
INFO:tensorflow:Saving dict for global step 30000: global_step = 30000, loss = 0.029794343, my_metric = 0.9904
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 30000: ./udf_log/model.ckpt-30000


In [18]:
acc_score = test_result["my_metric"]
print("\nTest acc: %g %%" % (acc_score*100))



Test acc: 99.04 %


In [21]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"image":mnist.test.images[:20]},
    num_epochs=1,
    batch_size=128,
    shuffle=False
)

In [22]:
predictions = estimator.predict(input_fn =predict_input_fn )
for i,p in enumerate(predictions):
    print("Prediction %s: %s" % (i+1,p["result"]))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./udf_log/model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction 1: 7
Prediction 2: 2
Prediction 3: 1
Prediction 4: 0
Prediction 5: 4
Prediction 6: 1
Prediction 7: 4
Prediction 8: 9
Prediction 9: 5
Prediction 10: 9
Prediction 11: 0
Prediction 12: 6
Prediction 13: 9
Prediction 14: 0
Prediction 15: 1
Prediction 16: 5
Prediction 17: 9
Prediction 18: 7
Prediction 19: 3
Prediction 20: 4
